# tugas 1 Visi Computer

|Name    | NPM   |
|--------|-------|
|M. Afkar Siddiq | 2008107010030|

In [1]:
#Berikut ini adalah code untuk menampilkan video live stream dari kamera 
#framenya diperkecil menjadi setengahnya
#untuk menutup frame, klik key 'q'

import cv2

cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    cv2.imshow('Input', frame)
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Berikut ini merupakan code yang menampilkan video live stream dari kamera
#kemudian anda dapat menggunakan mouse input dan membuat persegi di mana
#gambar pada persegi akan diubah menjadi citra negatif

import cv2
import numpy as np

def draw_rectangle(event, x, y, flags, params):
    global x_init, y_init, drawing, top_left_pt, bottom_right_pt
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        x_init, y_init = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            top_left_pt = (min(x_init, x), min(y_init, y))
            bottom_right_pt = (max(x_init, x), max(y_init, y))
            img[y_init:y, x_init:x] = 255 - img[x_init:x, y_init:y] #proses mengubah jadi negatif
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        top_left_pt = (min(x_init, x), min(y_init, y))
        bottom_right_pt = (max(x_init, x), max(y_init, y))
        img[y_init:y, x_init:x] = 255 - img[x_init:x, y_init:y] #proses mengubah jadi negatif

if __name__=='__main__':
    drawing = False
    top_left_pt, bottom_right_pt = (-1,-1), (-1,-1)
    
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
        
    cv2.namedWindow('Webcam')
    cv2.setMouseCallback('Webcam', draw_rectangle)
    
    while True:
        ret, frame = cap.read()
        img = cv2.resize(frame, (700, 500), interpolation=cv2.INTER_AREA)
        
        (x0,y0), (x1,y1) = top_left_pt, bottom_right_pt
        img[y0:y1, x0:x1] = 255 - img[y0:y1, x0:x1]
        
        cv2.imshow('Webcam', img)
        
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Tugasnya
Silakan anda bereksperimen dengan bagian persegi yang dipilih pada video live stream dari camera
Jika pada contoh bagian persegi diubah menjadi negatif,
maka pada tugas ini, anda bisa berkreasi, seperti mengubah warna, apply filter seperti blur,
gaussianblur, medianblur, atau menampilkan efek lainnya
atau anda bisa berkreasi pada keseluruhan gambar dengan menambah facemask, kumis, telinga kucing atau lainnya

In [67]:
#muka Blur

import cv2

faceDet = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while (True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceDet.detectMultiScale(gray)
    for (x,y,w,h) in faces:  
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),5)
        img[y:y+h, x:x+w] = cv2.GaussianBlur(img[y:y+h, x:x+w], (23, 23), 30)
    cv2.imshow("Blurred Img", img)
    if (cv2.waitKey(1) == ord('q')):
        break;
        
cap.release()
cv2.destroyAllWindows()   
        

In [8]:

#filter membuat hidung badut

import cv2
import numpy as np


#File cascade untuk mendeteksi wajah dan hidung
faceCascadeFilePath = 'haarcascade_frontalface_default.xml'
noseCascadeFilePath = "haarcascade_mcs_nose.xml"

faceCascade = cv2.CascadeClassifier(faceCascadeFilePath)
noseCascade = cv2.CascadeClassifier(noseCascadeFilePath)
 

 
#Ambil gambar hidung badut dari direktori
imgclown = cv2.imread('hidungbadut.png',-1)
 
#buat mask
orig_mask = imgclown[:,:,2]
 
#buat inverted mask untuk clown
orig_mask_inv = cv2.bitwise_not(orig_mask)
 
# Convert clown image ke BGR
# simpan ukuran original(digunakan kembali ketika re-sizing image)
imgclown = imgclown[:,:,0:3]
origclownHeight, origclownWidth = imgclown.shape[:2]
 

# buka video dari webcam
video_capture = cv2.VideoCapture(0)
 
while True:
    # Tangkap video feed
    ret, frame = video_capture.read()
 
    # buat greyscale image dari video feed
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    # Deteksi mukanya
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
 
   # iterasi setiap muka ditemukan
    for (x, y, w, h) in faces:
 
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
 
        # Mendeteksi hidung dari muka
        nose = noseCascade.detectMultiScale(roi_gray)
 
        for (nx,ny,nw,nh) in nose:
 
            # Membuat lebar kumis 3 kali lebar hidung
            clownWidth = nw
            clownHeight = clownWidth * origclownHeight / origclownWidth
 
            # Membuat clown di bawah hidung(di tengah)
            x1 = int(nx - (clownWidth/4))
            x2 = int(nx + nw + (clownWidth/4))
            y1 = int(ny + nh - (clownHeight/2))
            y2 = int(ny + nh + (clownHeight/2))
 
          
            if x1 < 0:
                x1 = 0
            if y1 < 0:
                y1 = 0
            if x2 > w:
                x2 = w
            if y2 > h:
                y2 = h
 
            # Re-calculate tinggi dan lebar clown image
            clownWidth = x2 - x1
            clownHeight = y2 - y1
 
            # Re-size original image dan masks sesuai dengan gambar clown
            clown = cv2.resize(imgclown, (clownWidth,clownHeight), interpolation = cv2.INTER_AREA)
            mask = cv2.resize(orig_mask, (clownWidth,clownHeight), interpolation = cv2.INTER_AREA)
            mask_inv = cv2.resize(orig_mask_inv, (clownWidth,clownHeight), interpolation = cv2.INTER_AREA)
 
 
            roi = roi_color[y1:y2, x1:x2]
 
            roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
 
            roi_fg = cv2.bitwise_and(clown,clown,mask = mask)
 
            dst = cv2.add(roi_bg,roi_fg)
 
            roi_color[y1:y2, x1:x2] = dst
 
            break
 
    # Tampilkan videonya
    cv2.imshow('Video', frame)
 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 

video_capture.release()
cv2.destroyAllWindows()